### An implementation of Time Series Classification on primate splice-junction data. The goal of this project is to classify intron-exon and exon-intron boundaries.
# Data Cleanup
First, we import all libraries used, and the data:

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib as plt
from random import choice

df = pd.read_csv('splice.data', names = ['Class', 'Name', 'Sequence'])
df.head()

,Class,Name,Sequence
0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGG...
1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGG...
2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGG...
3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGT...
4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTG...


Since the original data may have inconsistent spacing, we will strip any leading or tailing whitespace from each entry.

In [2]:
df['Class'] = df['Class'].str.strip()
df['Name'] = df['Name'].str.strip()
df['Sequence'] = df['Sequence'].str.strip()

For the purposes of our initial model, we will remove any instances in the "N" or Neither class. This will leave only the EI and IE class for analysis.

In [3]:
df = df[df.Class != 'N']
df['Class'].value_counts()

IE    768
EI    767
Name: Class, dtype: int64

There is one entry that has incomplete data; it can be removed.

In [4]:
df[df.Name == 'HUMALPI1-DONOR-42'].iloc[0, 2]
df = df[df.Name != 'HUMALPI1-DONOR-42']

'CACACAGGGCACCCCCTCANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'

The "Name" feature is unnecessary for the procedure, and can be dropped.

In [5]:
df = df.drop('Name', axis=1)
df.head()

,Class,Sequence
0,EI,CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...
1,EI,AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...
2,EI,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...
3,EI,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...
4,EI,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...


Mapping EI and IE to 0 and 1 respectively, makes it easier for our classifier later on.

In [6]:
df['Class'] = df['Class'].map({'EI': 0, 'IE': 1})
df.head()
df.tail()

,Class,Sequence
0,0,CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...
1,0,AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...
2,0,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...
3,0,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...
4,0,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...


,Class,Sequence
1530,1,AGCCTGGGCTGACCCCACGTCTGGCCACAGGCCCGCGTGCTGCCCC...
1531,1,CTGTCCTGTGGGTTCCTCTCACCCCCTCAGGCTGCTGGTCGTCTAC...
1532,1,ATGTTTAAACCTCGCGTTTCCTCCCCGCAGCTCTTGGGCAATGTGC...
1533,1,CTGTCCTGTGGGTTCCTCTCACCCTCTCAGGTTGCTGGTCGTCTAC...
1534,1,CATATGTATCTTTTTACCTTTTCCCAACAGCTCCTGGGCAACGTGC...


It is necessary to encode our nucleotides into numbers so their pattern and ordering can be analyzed. The bases will be encoded into 1, 2, 3, and 4 in alphabetical order. The ambiguous nucleotides, N, D, S, and R will be pseudorandomly chosen, based on the possible nucleotides it could be.

In [7]:
def translateSequence(sequence):
    baseDict = {'A':'1', 'C':'2', 'G':'3', 'T':'4', 'N':choice(['1', '2', '3', '4']),
               'D':choice(['1', '3', '4']), 'S':choice(['2', '3']), 'R':choice(['1', '3'])}
    newSequence = ''
    for base in sequence:
        newSequence += baseDict[base] 
    return newSequence

df['Sequence'] = [translateSequence(sequence) for sequence in df['Sequence']]
df.head()

,Class,Sequence
0,0,2213243214212133133221323132133424344221133322...
1,0,1312223223331332331331224321333431322221223222...
2,0,3133431133123422442222133132233431311323213423...
3,0,3332432344324334212144224332133414333323333244...
4,0,3242132222213342122213311243123431343422221422...


Finally, we will separate each columns into their own individual Numpy arrays. Note that each sequence of numbers is being split, forming a matrix. Each row in this matrix will contain a sequence that has been encoded into numbers.

In [8]:
y = np.asarray(df['Class'])
y
np.shape(y)
X = np.asarray([list(map(int, sequence)) for sequence in df['Sequence']])
X
np.shape(X)

array([0, 0, 0, ..., 1, 1, 1])

(1534,)

array([[2, 2, 1, ..., 2, 4, 3],
       [1, 3, 1, ..., 2, 3, 2],
       [3, 1, 3, ..., 1, 4, 3],
       ...,
       [1, 4, 3, ..., 2, 4, 3],
       [2, 4, 3, ..., 3, 1, 3],
       [2, 1, 4, ..., 2, 4, 3]])

(1534, 60)